# Third Step
This notebook takes the summarized paper from the second notebook to extract the relevant parameters from it. It uses the lmformatenforcer library to enforce the desired output

In [ ]:
from transformers import GenerationConfig, LlamaForCausalLM, LlamaTokenizer
from peft import PeftModel
import json, jsonlines
import torch

base_model= "meta-llama/Llama-2-70b-chat-hf"
use_perf = False

tokenizer = LlamaTokenizer.from_pretrained(base_model)
model = LlamaForCausalLM.from_pretrained(
    base_model,
    load_in_8bit=True,
    device_map="auto",
)
if use_perf:
    model = PeftModel.from_pretrained(
            model,
            '/home/ws/kg2371/lora_weights/llama2_70b_8bit_mof_fold0_4epoch',
            device_map="auto",
    )

In [ ]:
system_msg = """You are a chemistry assistant that generates tables related to MOF synthesis. If something is not provided within the context don't make anything up!

Please summarize the following details in a table from the given context:
    - Composite Name (Name of the formulated biohybrid enzyme-MOF composite)
    - MOF Name (Name of the enclosing MOF)
    - Linker Salt (Chemical Name of the linker salt used for immobilization)
    - Metal Salt (Chemical Name of the metal salt used for immobilization)
    - Enzyme Name (Name of the immobilized enzyme)
    - Organism Name (Name of the organism of origin of the immobilized enzyme)
    - Concentration Metal Salt (Concentration of the used metal salt solution)
    - Concentration Linker (Concentration of the used linker salt solution)
    - Concentration Enyzme  (Concentration of the used enzyme solution)
    - Reactants (Educts of the performed enzymatic Reaction)
    - Products (Products of the performed enzymatic Reaction)
    - Sequence of Components (Sequence of the addition of solutions during the formulation of the biohybrid enzyme-MOF-composite)

If any information is not provided or you are unsure, use "N/A".
If multiple conditions are provided for the same compound, use multiple rows to represent them. If multiple units or components are provided for the same factor (e.g. g and mol for the weight, multiple linker or metals, multiple temperature and reaction time, mixed solvents, etc.), include them in the same cell and separate by comma.

The table must have 12 columns, all in lowercase:| composite_name | mof_name | linker_salt | metal_salt | enzyme_name | organism_name | concentration_metal_salt | concentration_linker | concentration_enzyme | reactants | products | sequence_of_components |"""

example_input_1 = """Context:\nThe Metal-Organic Framework (MOF) used in this study is ZIF-8, which was synthesized by mixing 2-methylimidazole (24 mmol) and zinc acetate (8 mmol) with orange peel extract at room temperature within 30 minutes. The resulting OPP-MOF composite showed enhanced thermal stability, storage stability, and reusability for multiple cycles. The immobilized OPP-MOF was used for degradation of methylene blue (MB) and Congo red (CR) dye, and it was found to be more efficient and rapid. The Michaelis-Menten kinetics parameters were determined for OPP-MOF and the free form of peroxidase, and thermo-kinetic studies were carried out to determine the thermal stability in terms of thermal deactivation constant (kd), half-life (t1/2) and deactivation energy (Ea). The OPP-MOF composite was characterized using XRD, FT-IR, and SEM. The peroxidase enzyme was extracted from fresh unripe oranges (_Citrus sinensis_) and immobilized within the OPP-MOF composite. The OPP-MOF synthesis was carried out by mixing zinc acetate (8 mmol, 14 mL) with an enzyme extract solution (containing protein content of 1.2 mg/mL) of 2-methylimidazole water solution (24 mmol, 14 mL) and stirring at 200 rpm at 28 ± 2 degC."""

example_output_1 = f"""| composite_name | mof_name | linker_salt | metal_salt | enzyme_name | organism_name | concentration_metal_salt | concentration_linker | concentration_enzyme | reactants | products | sequence_of_components |
| OPP-MOF | ZIF-8 | zinc acetate | 2-methylimmidazole | orange peel peroxidase | Citrus sinensis | 8 mmol | 24 mmol | 1.2 mg/mL | Congo red, methylene blue | N/A | 1. 2-methylimmidazole, orange peel peroxidase, 2. zinc acetate |"""

example_input_2 = """The laccase@HKUST-1 biocomposite was synthesized via a biomimetic mineralization process, enhancing the catalytic activity of laccase by 1.5-fold compared to free laccase. The biocomposite consisted of laccase enzyme (E.C. 1.10.3.2.) and HKUST-1 metal organic framework (MOF), containing cofactor Ca2+. The MOF had a porous structure with a surface area of 6,426 m2/g and pore size of 0.54 nm. The laccase enzyme was immobilized within the MOF pores, forming a stable complex. The biocomposite was tested for its ability to degrade bisphenol A (BPA) and showed 100% degradation efficiency within 4 hours, with improved stability and activity under acidic conditions (pH 4.5), high temperatures (50°C), organic solvents (10% DMF), and continuous operation. The enzymatic reaction involved the oxidation of BPA by laccase, resulting in the formation of a catechol intermediate, which was further oxidized to form a quinone. The quinone was then reduced back to BPA, resulting in the degradation of the pollutant. The synergistic enhancement of substrate oxidation was due to the integration of laccase and HKUST-1 containing cofactor Ca2+, which improved the electron transfer efficiency and stability of the enzyme. The study demonstrates a feasible method to improve the activity and stability of laccase, enabling the complete removal of BPA from the environment. Laccase (from _Tranetes versicolar_, powder, 0.5 Umg-1, EC 1.10.3.2) and 2,2'-azino-bis(3-ethylbenzothiazoline-6-sulfonic acid) (ABTS, 98%) were purchased from Sigma-Aldrich Co. 1,3,5-benzenetricarboxylicacid (C9H0Co9, 98%) was obtained from Jinan Henghua Sci. & Tec. Co. Ltd. Cupric acetate monohydrate (CuCH3COO)2H2O2,0 > 98%) was from Sinopham (Shanghai, China) Chemical Reagent Co. Ltd. Bisphenol A (BPA: (CH3)2C(C6H4OH)2, 99%) was purchased from Aladdin. All other chemicals (analytical grade) were from Sinopham (Shanghai, China) Chemical Reagent Co. Ltd. and used without modification. The laccase@HKUST-1 biocomposite was fabricated using the biomimetic mineralization method. The method involved preparing a mixture A by adding cupric acetate monohydrate solution (87.5 mM, 250 μL) to a solution of laccase (2 mg, 20 μL). Solution B was obtained by dissolving BTC (25 mM, 250 μL) in an acetate buffer saline solution (pH 7.0, 200 mM). The solutions A and B were mixed and incubated at 30 °C for 8 h to form HKUST-1 protective coatings. The solids were separated by centrifugation at 6000 rpm for 10 min and washed three times with deionized water. The solids were then dried in a vacuum freeze dryer for 12 h and stored at 4 °C."""

example_output_2 = f"""| composite_name | mof_name | linker_salt | metal_salt | enzyme_name | organism_name | concentration_metal_salt | concentration_linker | concentration_enzyme | reactants | products | sequence_of_components |
| laccase@HKUST-1 | HKUST-1 | 1,3,5-benzenetricarboxylicacid | Cupric acetate monohydrate  | Laccase | N/A | N/A | N/A | N/A | BPA | N/A | 1. Cupric acetate monohydrate, Laccase, 2. 1,3,5-benzenetricarboxylic acid|"""


input_text = """Context:\n  The nanoscale metal organic framework (NMOF) NH2-ML-53(Al) was synthesized via a one-pot, green procedure using the metal salt anhydrous aluminum chloride powder (AlCl3, 99%) and the linker salt 2-amino-terephthalic acid (NH2-BDC). The reaction was stirred for 24 hours at 25°C, and the resulting NMOF was washed with deionized water and dried at 60°C for 12 hours, exhibiting a surface area of 1500 m2/g and a pore size of 0.7 nm. The immobilization of laccase enzyme onto the NMOF was achieved by coprecipitation with a molar ratio of 1:1 for laccase:NMOF, followed by stirring for 2 hours at 25°C and centrifugation. The immobilized laccase was reused for 10 cycles, retaining 80% of its initial activity after each cycle, and its stability was evaluated at different pH values (4-6) and temperatures (25-50°C). The NMOF support was found to be stable at pH 4-6, and the laccase enzyme, obtained from Trametes versicolor, had a specific activity of >=0.5 U mg-1. The enzymatic reaction was carried out at 25°C and pH 4.5, using 1 mM ABTS as the substrate, and the reaction was monitored by measuring the increase in absorbance at 420 nm. The resulting NMOF-immobilized laccase [NH2-MIL-53(Al)/laccase] had a specific activity of >=0.5 U mg-1, and the enzymatic reaction was carried out at 25°C and pH 4.5, using 1 mM ABTS as the substrate. The Michaelis-Menten constant (_K_m) and maximum rate of reaction (_V_max) of free and immobilized laccase were determined by varying the substrate (ABTS) concentrations from 0.5 mM to 1.0 mM in acetate buffer solution (pH ~ 4.6) at 25°C. The kinetic parameters were calculated from the Lineweaver-Burk plot, which showed a linear relationship between 1/V and 1/[S]. The NMOF-immobilized laccase [ MOF-Name: NH2-MIL-53(Al)/laccase] had a specific activity of >=0.5 U mg-1, and the enzymatic reaction was carried out at 25°C and pH 4.5, using 1 mM ABTS as the substrate. The reaction was monitored by measuring the increase in absorbance at 420 nm, and the resulting product was separated by centrifugation. The effects of pH and temperature on free and immobilized laccase were studied. For the pH study, the enzyme activity was measured in acetate buffer at different pH values (3-7) at room temperature. Immobilized laccase was stored at 4°C for seven weeks, and its activity was examined at 7 day intervals in acetate buffer solution. The activity remained stable throughout the storage period. The loading capability of enzymes into NMOFs was evaluated by using different amounts of laccase in the one-pot synthesis. NH2-MIL-53(Al) was synthesized in water at room temperature. Solution 1: 75.12 mg (0.253 mmol) NH2-BDC and 28 mg (0.7 mmol) NaOH in 4 mL distilled water with magnetic stirring at 400 rpm. Solution 2: 100 mg AlCl3 in 4 mL distilled water. Solution 2 was added dropwise to solution 1, forming a yellow solid. After 12 h, the product was separated by centrifugation, washed 3x with distilled water, and dried at 55°C in a vacuum oven. NH2-MIL-53(Al) was prepared using the procedure described above. Solutions 1 and 2 were prepared as above, with solution 2 added dropwise to solution 1 with stirring. After stirring for 1 h, a fixed amount of laccase (100 mg) in distilled water (6 mL) was added dropwise to the mixture and stirred for a further 12 h. The resultant yellow product was separated from the reaction mixture by centrifugation and washed three times with distilled water. residual product NH2-ML53(Al)/laccase was dried at 55 degC in a vacuum oven for further characterization. The optimal conditions for the immobilization process were found to be 30°C, pH 5.0, and 100 mM NaCl, with a maximal activity of 51.7 U/mg and a half-life of 4.2 h. The immobilized laccase was able to degrade phenol and bisphenol A, with a degradation rate constant of 0.043 min-1 and 0.037 min-1, respectively. The immobilized laccase was also stable in the presence of metal ions, retaining 80% of its initial activity in the presence of 100 mM Cu2+ and 50 mM H2SO4. The surface area analysis of NH2-MIL-53(Al) and NH2-MIL-53(Al)/lactase was performed using Microporous BET analysis. The surface area of NH2-MIL-53(Al) was found to be 345.8 m2 g-1, which is high, leading to a high amount of enzyme loading. After enzyme loading, the surface area of NH2-MIL-53(Al)/lactase was observed to be 116.1 m2 g-1, indicating that the laccase was loaded into the NMOF, leading to a decrease in surface area. The pore diameter of the NMOF was 0.545 nm, which decreased to 0.277 nm after laccase immobilization, due to the NMOF pores being blocked by laccase."""

messages = [
    {"role": "system", "content": system_msg},
    {"role": "user", "content": example_input_1},
    {"role": "assistant", "content": example_output_1},
    {"role": "user", "content": example_input_2},
    {"role": "assistant", "content": example_output_2},
    {"role": "user", "content": input_text}
]
prompt = tokenizer.decode(tokenizer.apply_chat_template(messages, return_tensors="pt")[0],skip_special_tokens=False)
prompt += " | composite_name | mof_name | linker_salt | metal_salt | enzyme_name | organism_name | concentration_metal_salt | concentration_linker | concentration_enzyme | reactants | products | sequence_of_components |\n|"
print(f"Number of tokens for request: {tokenizer(prompt,return_tensors='pt').to('cuda')['input_ids'].shape[1]}")

In [ ]:
from tqdm import tqdm
from lmformatenforcer.integrations.transformers import build_token_enforcer_tokenizer_data
from lmformatenforcer.integrations.transformers import generate_enforced
from lmformatenforcer import RegexParser

responses = []

tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer_data = build_token_enforcer_tokenizer_data(tokenizer)

answer_regex = r"[a-zA-Z0-9 / \^ \- \. , \@ \( \/ \)]+\|[a-zA-Z0-9 / \^ \- \. , \@ \( \/ \)]+\|[a-zA-Z0-9 / \^ \- \. , \@ \( \/ \)]+\|[a-zA-Z0-9 / \^ \- \. , \@ \( \/ \)]+\|[a-zA-Z0-9 / \^ \- \. , \@ \( \/ \)]+\|[a-zA-Z0-9 / \^ \- \. , \@ \( \/ \)]+\|[a-zA-Z0-9 / \^ \- \. , \@ \( \/ \)]+\|[a-zA-Z0-9 / \^ \- \. , \@ \( \/ \)]+\|[a-zA-Z0-9 / \^ \- \. , \@ \( \/ \)]+\|[a-zA-Z0-9 / \^ \- \. , \@ \( \/ \)]+\|[a-zA-Z0-9 / \^ \- \. , \@ \( \/ \)]+\|[a-zA-Z0-9 / \^ \- \. , \@ \( \/ \)]+\|"

parser = RegexParser(answer_regex)


model.eval()
with torch.no_grad():
    inputs = tokenizer(prompt, return_tensors='pt', add_special_tokens=False, return_token_type_ids=False, padding=True).to('cuda')
    generate_kwargs = dict(
        inputs,
        max_new_tokens=512,
        do_sample=True,
        top_p=0.95, # Only consider the possibilities that equal or exceed this value
        top_k=50, # how many next token options are considered
        temperature=0.1, # randomness, flattening of probability values
        num_beams=1,
        output_scores=True,
        return_dict_in_generate=True
    )
    response = generate_enforced(model,tokenizer_data,parser,**generate_kwargs)
    response_extracted = tokenizer.decode(response['sequences'][0], skip_special_tokens=False)
    response_extracted = response_extracted.replace(prompt,'')
    response_extracted = response_extracted.replace(tokenizer.eos_token,"")
    response_extracted = response_extracted.strip()
model.train()
print(response_extracted)

In [ ]:
import pandas as pd
pd.set_option("display.max_rows", None)


display(pd.DataFrame(response.enforced_scores))